In [1]:
import os
import glob
import altair as alt
import pandas as pd
import numpy as np
import warnings
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from pathlib import Path
import os
import plotly
from tqdm import tqdm
pd.set_option('display.max_rows', 500)

from bs4 import BeautifulSoup
import requests as req
from time import sleep
import pandas as pd
import os
import time
from datetime import datetime as dt
from tqdm import tqdm
#from tqdm import tqdm


# configure pandas
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"
    )
}

In [2]:
nfl_date = "20221129"
path_base = Path(os.getcwd()).parents[1]
print(path_base)
cols = ["Date", "Opponent", "Result", "W/L", "Div", "Spread", "Total", "Money"]
file_path = f"{path_base}/data/team_ranks/_nfl_stat*{nfl_date}.csv"
file_paths = glob.glob(os.path.join(file_path))
pds = list()
pbar = tqdm(file_paths)
for file_path in pbar:
    pbar.set_description(file_path)
    df = pd.read_csv(file_path)
    pds.append(df)
df = pd.concat(pds)
df["_stat"] = df.url.str.split("/", expand=True)[3]
df["Stat"] = df._stat.str.split("-").str.join(" ").str.title()
df["Stat"] = df.Stat.str.replace("Per Game", "Pg").str.replace("Opponent", "Opp")\
.str.replace("Offensive", "Off")\
.str.replace("Defensive", "Def")\
.str.replace("Touchdowns", "Td")\
.str.replace("Rushing", "Run")\
.str.replace("Passing", "Pass")\
.str.replace("Average", "Avg")\
.str.replace("Passer", "Psr")\
.str.replace("Interceptions", "Int")
df["_team"] = df.Team.str.lower().str.split(" ").str.join("-")
stat_matchup_map = {
    "points-per-game":"opponent-points-per-game",
    "points-per-play":"opponent-points-per-play",
    "passing-yards-per-game":"opponent-passing-yards-per-game",
    # sacks-per-game -> defense
    "qb-sacked-per-game":"sacks-per-game",
    "interceptions-per-game":"interceptions-thrown-per-game",
    "1st-half-points-per-game":"opponent-1st-half-points-per-game",
    "2nd-half-points-per-game":"opponent-2nd-half-points-per-game",
    "rushing-yards-per-game":"opponent-rushing-yards-per-game",
    "passing-yards-per-game":"opponent-passing-yards-per-game",
    "rushing-touchdowns-per-game":"opponent-rushing-touchdowns-per-game",
    "passing-touchdowns-per-game":"opponent-passing-touchdowns-per-game",
    "yards-per-game":"opponent-yards-per-game",
    "yards-per-play":"opponent-yards-per-play",
    "yards-per-pass-attempt":"opponent-yards-per-pass-attempt",
    "yards-per-rush-attempt":"opponent-yards-per-rush-attempt",
    "red-zone-scores-per-game":"opponent-red-zone-scores-per-game",
    "red-zone-scoring-pct":"opponent-zone-scoring-pct",
    "penalties-per-game":"opponent-penalties-per-game",
    "offensive-touchdowns-per-game":"opponent-offensive-touchdowns-per-game",
    "defensive-touchdowns-per-game":"opponent-defensive-touchdowns-per-game",
    "average-team-passer-rating":"opponent-average-team-passer-rating",
    "seconds-per-play":"opponent-seconds-per-play",
    "giveaways-per-game":"takeaways-per-game"
}
df = df.drop(columns=["Unnamed: 0"])

/notebooks/chicago_bears_nerd


/notebooks/chicago_bears_nerd/data/team_ranks/_nfl_stat_opponent_penalties_per_play_20221129.csv: 100%|██████████| 226/226 [00:00<00:00, 291.18it/s]                     


In [3]:
# 557 == week 11
url = "https://www.teamrankings.com/nfl/schedules/season/?week=559"
page = req.get(url, headers=headers).text
soup = BeautifulSoup(page, "html.parser")
table = soup.find("table")
df_schedule = pd.read_html(str(table), displayed_only=False)[0]
df_schedule.columns = ["teams", "time", "location"]

In [4]:
df_schedule["home"] = df_schedule.teams.str.split("@", expand=True)[1]

In [5]:
df_schedule.home = df_schedule.home.str.strip(" ")
df_schedule["away"] = np.where(df_schedule.teams.str.contains("vs."), df_schedule.teams.str.split("vs.", expand=True)[0], df_schedule.teams.str.split("@", expand=True)[0])
df_schedule.away = df_schedule.away.str.strip(" ")
df_schedule["_home"] = df_schedule.home.str.lower().str.split(" ").str.join("-")
df_schedule["_away"] = df_schedule.away.str.lower().str.split(" ").str.join("-")

team_matchup_map = df_schedule.set_index("_home")._away.to_dict()

In [6]:
def find_matchup_rank(df, team_matchup_map, stat_matchup_map):
    df["matchup_key"] =  df._team +"-"+ df._stat
    df["matchup_team"] = df._team.map(team_matchup_map).str.title()
    df["matchup_key_lookup"] = df.matchup_team.str.lower() +"-"+ df._stat.map(stat_matchup_map)
    matchup_dict_data = {k:v for k,v in df.set_index("matchup_key")["Rank"].to_dict().items() if not pd.isna(k)}
    df["matchup_rank"] = df.matchup_key_lookup.map(matchup_dict_data)
    
    matchup_dict_data = {k:v for k,v in df.set_index("matchup_key")["2022"].to_dict().items() if not pd.isna(k)}
    df["matchup_value"] = df.matchup_key_lookup.map(matchup_dict_data)
    return df


In [7]:
stat_matchup_map.update({v:k for k,v in stat_matchup_map.items()})
df = find_matchup_rank(df, team_matchup_map, stat_matchup_map)

In [8]:
df["matchup_diff"] = df.matchup_rank - df.Rank

In [9]:
df_heat = df[~df.matchup_rank.isna()][["Team", "matchup_team", "Stat", "2022", "Rank", "matchup_rank", "matchup_value", "matchup_diff"]]
df_heat["matchup"] = df_heat.matchup_team +" @ " +df_heat.Team
df_heat["_matchup_diff"] = np.where(abs(df_heat.matchup_diff)>15, df_heat.matchup_diff, None)

df_heat["home_rank_win_count"] = np.where(df_heat.Rank>df_heat.matchup_rank, 1, 0)
df_heat["away_rank_win_count"] = np.where(df_heat.Rank>df_heat.matchup_rank, 0, 1)
df_heat_agg = df_heat.groupby("matchup").agg(away_rank_mean=("matchup_rank", "mean"), 
                               home_rank_mean=("Rank", "mean"), 
                               home_rank_win_count=("home_rank_win_count", "sum"), 
                               away_rank_win_count = ("away_rank_win_count", "sum")
                              )
df_heat_agg.away_rank_mean = df_heat_agg.away_rank_mean.round(2)
df_heat_agg.home_rank_mean = df_heat_agg.home_rank_mean.round(2)


In [10]:
df_heat_agg

,away_rank_mean,home_rank_mean,home_rank_win_count,away_rank_win_count
matchup,,,,
Buffalo @ New England,9.26,14.31,30,12
Cleveland @ Houston,17.45,23.55,32,10
Denver @ Baltimore,16.90,12.12,18,24
Green-Bay @ Chicago,19.26,20.86,24,18
Indianapolis @ Dallas,20.02,9.40,11,31
Jacksonville @ Detroit,13.86,16.83,21,21
Kansas-City @ Cincinnati,10.86,11.90,22,20
La-Chargers @ Las Vegas,19.12,18.69,17,25
Miami @ San Francisco,14.76,9.57,12,30


In [35]:
chart_title = "Matchup Finder Week 13"
x_cfgs = {"shorthand":"matchup:O", "title": " "}
y_cfgs = {"shorthand":"Stat", "title":" "}
# background etc.
primary_color = "#0B162A" 
# font etc.
secondary_color = "#c83803"

chart = alt.Chart(df_heat, width=1400, height=2000, title=chart_title).mark_rect()\
.encode(
    y=alt.Y(**y_cfgs,  axis=alt.Axis(labelLimit=600)),
    x=alt.X(**x_cfgs, axis=alt.Axis(labelLimit=600)),
    color=alt.Color(
        "_matchup_diff:Q", 
        scale=alt.Scale(range=['red','white','green']), 
        legend=alt.Legend(title='Matchup Diff'),
    ),
    tooltip=[
        alt.Tooltip("matchup", title="Game"),
        alt.Tooltip("Stat", title="Stat"),
        alt.Tooltip("Rank", title="Home Team Rank"),
        alt.Tooltip("2022", title="Home Team Stat Value"),
        alt.Tooltip("matchup_rank", title="Away Team Rank"),
        alt.Tooltip("matchup_value", title="Away Team Stat Value"),
        alt.Tooltip("matchup_diff", title="Rank Diff"),        
    ]
)

text = chart.mark_text(baseline='middle', fontSize=15).encode(
    text='matchup_diff:Q',
    color=alt.value('white')
    )


plot = (chart + text)\
.configure(background=primary_color)\
.configure_axis(gridColor="grey", gridDash=[6,4])\
.configure_axisLeft(labelColor=secondary_color, titleColor=secondary_color, labelFontSize=20, titleFontSize=20)\
.configure_axisBottom(labelColor=secondary_color, titleColor=secondary_color,labelFontSize=20, titleFontSize=20, labelLineHeight=500)\
.configure_title(color=secondary_color, fontSize=35)\
.configure_legend(labelColor=secondary_color, titleColor=secondary_color, strokeColor='gray', fillColor=primary_color, padding=10,cornerRadius=10,orient='top-right', labelFontSize=15, titleFontSize=15)
plot.save("chart_json/matchup_finder.json")

plot

alt.LayerChart(...)